In [1]:
import pandas as pd

In [2]:
def retro_dictify(frame):
    d = {}
    for row in frame.values:
        here = d
        for elem in row[:-2]:
            if elem not in here:
                here[elem] = {}
            here = here[elem]
        here[row[-2]] = row[-1]
    return d

def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return list(frame.values.squeeze())
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
    return d

def hierarchy_flat(df, hierarchy_cols = None):
    if hierarchy_cols is None:
        hierarchy_cols = df.select_dtypes('object').nunique().index.tolist()
    df = df[hierarchy_cols]
    df = df.sort_values(hierarchy_cols).drop_duplicates()
    df = df.reset_index(drop = True)
    return df

def hierarchy_nested(df, hierarchy_cols = None):
    df_flat = hierarchy_flat(df, hierarchy_cols)
    return recur_dictify(df_flat)

def generate_schema(df):
    str_col_mapper = {}
    num_col_mapper = {}
    date_col_mapper = {}
    cols = df.columns.tolist()
    num_cols = df.select_dtypes("number").columns.tolist()
    str_cols = df.select_dtypes("object").columns.tolist()
    date_cols = df.select_dtypes("datetime").columns.tolist()
    for col in str_cols:
        str_col_mapper[col] = df[col].fillna("").sort_values().unique().tolist()
    for col in num_cols:
        num_col_mapper[col] = {
            "min": float(df[col].min()),
            "max": float(df[col].max()),
            "is_na": bool(df[col].isna().any()),
        }
    for col in date_cols:
        date_col_mapper[col] = {"date_freq": None, "date_format": None}
    data_schema = {
        "columns": cols,
        "num_cols": num_cols,
        "str_cols": str_cols,
        "date_cols": date_cols,
        "str_col_metadata": str_col_mapper,
        "num_col_metadata": num_col_mapper,
        "date_col_metadata": date_col_mapper,
    }
    return data_schema

In [3]:
df = pd.read_csv("flights.csv")

In [4]:
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,date
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z,2013-01-01
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z,2013-01-01
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z,2013-01-01
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z,2013-01-01
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z,2013-01-01


In [7]:
data_schema = generate_schema(df.head(1000))

In [8]:
data_schema

{'columns': ['year',
  'month',
  'day',
  'dep_time',
  'sched_dep_time',
  'dep_delay',
  'arr_time',
  'sched_arr_time',
  'arr_delay',
  'carrier',
  'flight',
  'tailnum',
  'origin',
  'dest',
  'air_time',
  'distance',
  'hour',
  'minute',
  'time_hour',
  'date'],
 'num_cols': ['year',
  'month',
  'day',
  'dep_time',
  'sched_dep_time',
  'dep_delay',
  'arr_time',
  'sched_arr_time',
  'arr_delay',
  'flight',
  'air_time',
  'distance',
  'hour',
  'minute'],
 'str_cols': ['carrier', 'tailnum', 'origin', 'dest', 'time_hour', 'date'],
 'date_cols': [],
 'str_col_metadata': {'carrier': ['9E',
   'AA',
   'AS',
   'B6',
   'DL',
   'EV',
   'F9',
   'FL',
   'HA',
   'MQ',
   'UA',
   'US',
   'VX',
   'WN'],
  'tailnum': ['N0EGMQ',
   'N11107',
   'N11119',
   'N11189',
   'N11193',
   'N11194',
   'N11206',
   'N11536',
   'N11544',
   'N11551',
   'N11565',
   'N12157',
   'N12216',
   'N12218',
   'N12225',
   'N12540',
   'N12567',
   'N12996',
   'N13113',
   'N13123',

In [9]:
hierarchy_flat_df = hierarchy_flat(df.head(1000))

In [10]:
hierarchy_flat_df

,carrier,tailnum,origin,dest,time_hour,date
0,9E,N602LR,JFK,CVG,2013-01-02T00:00:00Z,2013-01-01
1,9E,N604LR,JFK,MSP,2013-01-01T20:00:00Z,2013-01-01
2,9E,N836AY,JFK,PHL,2013-01-02T01:00:00Z,2013-01-01
3,9E,N8409N,JFK,SYR,2013-01-01T20:00:00Z,2013-01-01
4,9E,N8444F,JFK,CLE,2013-01-02T00:00:00Z,2013-01-01
...,...,...,...,...,...,...
995,WN,N905WN,LGA,MKE,2013-01-02T00:00:00Z,2013-01-01
996,WN,N957WN,EWR,DEN,2013-01-01T14:00:00Z,2013-01-01
997,WN,N957WN,EWR,MDW,2013-01-02T00:00:00Z,2013-01-01
998,WN,N961WN,EWR,BWI,2013-01-02T11:00:00Z,2013-01-02


In [11]:
hierarchy_nested_dict = hierarchy_nested(df.head(1000))

In [12]:
hierarchy_nested_dict

{'9E': {'N602LR': {'JFK': {'CVG': {'2013-01-02T00:00:00Z': '2013-01-01'}}},
  'N604LR': {'JFK': {'MSP': {'2013-01-01T20:00:00Z': '2013-01-01'}}},
  'N836AY': {'JFK': {'PHL': {'2013-01-02T01:00:00Z': '2013-01-01'}}},
  'N8409N': {'JFK': {'SYR': {'2013-01-01T20:00:00Z': '2013-01-01'}}},
  'N8444F': {'JFK': {'CLE': {'2013-01-02T00:00:00Z': '2013-01-01'},
    'IAD': {'2013-01-01T20:00:00Z': '2013-01-01'}}},
  'N8515F': {'JFK': {'RDU': {'2013-01-01T21:00:00Z': '2013-01-01'}}},
  'N8598B': {'JFK': {'BWI': {'2013-01-02T01:00:00Z': '2013-01-01'}}},
  'N8611A': {'JFK': {'RDU': {'2013-01-02T00:00:00Z': '2013-01-01'}}},
  'N8631E': {'JFK': {'ROC': {'2013-01-01T20:00:00Z': '2013-01-01'}}},
  'N8869B': {'JFK': {'PIT': {'2013-01-02T13:00:00Z': '2013-01-02'}}},
  'N8894A': {'JFK': {'CHS': {'2013-01-01T23:00:00Z': '2013-01-01'}}},
  'N8946A': {'EWR': {'CVG': {'2013-01-02T11:00:00Z': '2013-01-02'}}},
  'N8968E': {'JFK': {'PHL': {'2013-01-01T21:00:00Z': '2013-01-01'}}},
  'N902XJ': {'JFK': {'PHL': {'201

#### __Note:__

* `hierarchy_nested` and `hierarchy_flat` takes `hierarchy_cols` as arg. This is a ordered list of hierarchical columns. Pass this if the information is already know. If the value is None, the this function will select all the string columns, will count the unique values in each string column, order the data set in the order of number of unique values in each string column, drop the duplicate rows and will create a dataframe or nested dict depending on the function.
* The last element or leaf node of the tree will be either a atomic data type of list of atomic data type. if the leaf is atomic data type (i.e. float, string, int or bool), then it means that there is only one element present in that level of hierarchy. if the leaf node is list, that means there are multiple values present in that level.